In [1]:
import sys
sys.path.append("..")

### 支持的数据集

In [ ]:
import recstudio.data as recdata
print(recdata.supported_dataset)

: 

### 入门级

In [3]:
from recstudio import quickstart

quickstart.run(model='SASRec', dataset='ml-100k', gpu=[2])

[2022-06-02 16:53:58] INFO Global seed set to 2022
[2022-06-02 16:53:58] INFO dataset is read from ../datasets/ml-100k.
/data1/home/huangxu/miniconda3/lib/python3.9/site-packages/pandas/io/parsers/readers.py:586: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  return _read(filepath_or_buffer, kwds)
[2022-06-02 16:53:58] INFO 
Dataset Info: 

item information: 
field        item_id      movie_title  release_year class        
type         token        token_seq    token        token_seq    
##           1683         2653         74           20           
user information: 
field        user_id      age          gender       occupation   
type         token        token        token        token        
##           944          62           3            22           
interaction information: 
field        user_id      item_id      rating       timestamp    
type         token        token        float        f

### 基础级

In [4]:
import torch
from recstudio.ann import sampler
from recstudio.data import dataset
from recstudio.model.basemodel import BaseRetriever
from recstudio.model import scorer, loss_func as loss

In [ ]:
ml_1m_data = dataset.MFDataset(name='ml-1m')

trn, val, tst = ml_1m_data.build(split_ratio=[0.7, 0.2, 0.1])

bpr = BaseRetriever(
    item_encoder = torch.nn.Embedding(trn.num_items, 64, 0),
    query_encoder = torch.nn.Embedding(trn.num_users, 64, 0),
    scorer = scorer.InnerProductScorer(),
    loss = loss.BPRLoss(),
    sampler = sampler.UniformSampler(trn.num_items-1)
)

bpr.fit(trn, val, negative_count=1)

bpr.evaluate(tst)